# ChatGPT Investigates Twitter's Tweet Sentiment


## Introduction :-
### Welcome to this light-hearted notebook where we will embark on a detective journey with ChatGPT to investigate the sentiment of tweets from the infamous world of Twitter. With a touch of humor, we'll explore how ChatGPT can assist us in deciphering the emotional undertones of tweets.

## Case Brief:
### Our mission is to analyze the sentiment of tweets and determine whether they are positive, negative, or neutral and also find out the specific context that will make that tweet positive, negative or neutral. ChatGPT will serve as our trusty sidekick, helping us make sense of the vast sea of tweets and uncover any hidden emotions lurking in the Twitterverse.



### Importing ChatGPT Libraries

In [ ]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['open_ai_key']
# print(openai.api_key)

### Importing Basic Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
from sklearn.model_selection import train_test_split

/home/aryan/miniconda3/envs/ML/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
df_train = pd.read_csv('/home/sahib/Desktop/Extra/train.csv')
df_test = pd.read_csv('/home/sahib/Desktop/Extra/test.csv')
df_submission = pd.read_csv('/home/sahib/Desktop/Extra/sample_submission.csv')
df_sub_73 = pd.read_csv('/home/sahib/Desktop/Extra/submission_73.5.csv')

In [ ]:
df_test.head()

,textID,text,sentiment
0,f87dea47db,Last session of the day http://twitpic.com/67ezh,neutral
1,96d74cb729,Shanghai is also really exciting (precisely -...,positive
2,eee518ae67,"Recession hit Veronique Branquinho, she has to...",negative
3,01082688c6,happy bday!,positive
4,33987a8ee5,http://twitpic.com/4w75p - I like it!!,positive


In [ ]:
df_train.head()

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [ ]:
df_train.shape

(27481, 4)

### Split the Data into Train and Test

In [ ]:
train, test = train_test_split(df_train, stratify=df_train['sentiment'].tolist(), test_size=0.01, random_state=42)

In [ ]:
test = test.reset_index(drop=True)

In [ ]:
test.head()

,textID,text,selected_text,sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral


### Prompt Engineering

In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [ ]:
delimiter = "####"
system_message = f"""
consider yourself as Human working on positive , negative or neural text classifier having great
deal of english language. Now given a text delimited with {delimiter} characters.
Highlight the parts of text which make it positive or negative or neutral.
Output should be in python dictionary format with keys being sentiment, part_of_text
and the values for sentiment and part_of_text should be single output string and should not have list of outputs for better understanding see examples bellow
where part_of_text contains only that words that have high similarity with the sentiments giving out different parts
of text which make it profane, vulgar, or offensive
Note that when sentiment is neutral, part_of_text simply contains user message and part_of_text contains only one index size

So, I need to train on these below examples
there will be 3 columns input_text,text_extracted,labels these columns will have values separated by comma
and their descriptions are as follows
input_text = actual text data
text_extracted = text which actually provides emotions to the original text
labels = labels/sentiment assigned to extracted text

1. you are stupid , stupid ,negative
2. Sooo SAD I will miss you here in San Diego!!!, Sooo SAD, negative
3. my boss is bullying me..., bullying me, negative
4. what interview! leave me alone, leave me alone, negative
5. Sons of bitch why couldn`t they put them on the releases we already bought, Sons of bitch, negative

6. A little happy for the wine jeje ok it`sm my free time so who cares jaja i love this day, A little happy fo, positive
7. lucky kid...i so wanna see loserville  pity im in oz...., lucky, positive
8. Happy Mothers day to all you Mums out there, Happy Mothers day to all you Mums out there, positive
9. Thank you, Afrin Nasal Spray! Also, I got a giant teacup tonight!, Thank you,, positive
10. Packing up for the trip to Ascari track, thank you Bacardi, thank you Bacardi, positive

11. yeah, it`s a bit 80s silly at points but i love it, yeah, it`s a bit 80s silly at points but i love it, neutral
12. It's gonna be a super long day, It`s gonna be a super long day, neutral
13. I'm soooooooo hungry right now. DH is still not home, I`m soooooooo hungry right now. DH is still not home., neutral
14. Where`d the songs go on the site I want 'Do You' on this computer too, Where`d the songs go on the site I want 'Do You' on this computer too, neutral
15. fighting again, fighting again, neutral
"""

In [ ]:
user_message = f"""\
 its cool. i ate too much ice cream
"""
messages =  [
{'role':'system',
 'content': system_message},
{'role':'user',
 'content': f"{delimiter}{user_message}{delimiter}"},
]
response = get_completion_from_messages(messages)
print(response)

{'sentiment': 'positive', 'part_of_text': 'its cool. i ate too much ice cream'}


In [ ]:
test['predicted_text'] = None
test['predicted_sentiment'] = None

### Note :-
### In our quest to explore the realm of GPT-powered language generation, we find ourselves relying on the Time library as our trusty companion. With our current limitations in GPT subscription, we embrace the art of patience, gracefully waiting for a minute before unleashing the power of three requests simultaneously. Like a well-choreographed symphony, we synchronize our actions with time, ensuring a harmonious flow in our interactions with GPT.

In [1]:
from time import sleep
for j,i in enumerate(test.index):
    if i%3==0:
        sleep(60)          # To manage rate limits and stay within the maximum number of requests per minute without a subscription
    user_message = test['text'][i]
    print(f'--->{user_message}\n')
    messages =  [
    {'role':'system',
     'content': system_message},
    {'role':'user',
     'content': f"{delimiter}{user_message}{delimiter}"},
    ]
    response = get_completion_from_messages(messages)
    try:
        response = json.loads(response.replace("'", '"'))
        print(j,response)
        print('\n')
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", j,e)
        continue
    selected_text = response.get('part_of_text')
    selected_sentiment = response.get('sentiment')
#     print(selected_text)
    test['predicted_text'][j] = selected_text
    test['predicted_sentiment'][j] = selected_sentiment
#     break

In [ ]:
test.head(221)

,textID,text,selected_text,sentiment,predicted_text,predicted_sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive,excited to see,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative,doesn`t work,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive,Aww bless her. She`s one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral,bored,neutral
...,...,...,...,...,...,...
216,aea00d01f6,i`m not being mean,i`m not being mean,positive,i`m not being mean,neutral
217,d5c1749c3a,my teeth and head hurts,my teeth and head hurts,negative,teeth and head hurts,negative
218,94b2a982ef,umm..nope not anymoree,umm..nope not anymoree,neutral,umm..nope not anymoree,neutral
219,72dffbd0ef,"I will, n btw happy mothers day sissy","I will, n btw happy mothers day sissy",positive,None,None


In [ ]:
good_test = test[0:219]
good_test = good_test.dropna()
good_test

,textID,text,selected_text,sentiment,predicted_text,predicted_sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive,excited to see,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative,doesn`t work,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive,Aww bless her. She`s one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral,bored,neutral
...,...,...,...,...,...,...
214,8b3a2058c5,my cousin is in jail for shoplifting and drug...,im upset,negative,"cousin is in jail for shoplifting and drugs, u...",negative
215,d240f7c6d8,Good Morning Hows u ?,Good Morning,positive,Good Morning Hows u ?,neutral
216,aea00d01f6,i`m not being mean,i`m not being mean,positive,i`m not being mean,neutral
217,d5c1749c3a,my teeth and head hurts,my teeth and head hurts,negative,teeth and head hurts,negative


In [2]:
for i in good_test.index:
    print(test['predicted_text'][i])

### Jaccard Similarity:- The tool we rely on to unveil the hidden connections and measure the overlap between sets of words, guiding us through the vast ocean of textual data.

In [ ]:
def Jaccard_Similarity(sen1, sen2):
    words_sen1 = set(sen1.lower().split())
    words_sen2 = set(sen2.lower().split())

    intersection = words_sen1.intersection(words_sen2)

    union = words_sen1.union(words_sen2)

    return float(len(intersection))/len(union)

In [3]:
per_list = []
for j,i in enumerate(good_test.index):
    sen1 = test['selected_text'][i]
    sen2 = test['predicted_text'][i]

    percentage = Jaccard_Similarity(sen1, sen2)
    print(percentage)
    per_list.append(percentage)

### Accuracy (when we use 15 statements in prompt)

In [ ]:
average = sum(per_list) / len(per_list)
print(average)

0.5447867869060422


## Test on 219 datasets

In [ ]:
good_test= good_test[:219]
good_test

,textID,text,selected_text,sentiment,predicted_text,predicted_sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive,excited to see,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative,doesn`t work,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive,Aww bless her. She`s one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral,bored,neutral
...,...,...,...,...,...,...
214,8b3a2058c5,my cousin is in jail for shoplifting and drug...,im upset,negative,"cousin is in jail for shoplifting and drugs, u...",negative
215,d240f7c6d8,Good Morning Hows u ?,Good Morning,positive,Good Morning Hows u ?,neutral
216,aea00d01f6,i`m not being mean,i`m not being mean,positive,i`m not being mean,neutral
217,d5c1749c3a,my teeth and head hurts,my teeth and head hurts,negative,teeth and head hurts,negative


### Prompt Engineering

In [ ]:
def get_completion_from_messages(messages,
                                 model="gpt-3.5-turbo",
                                 temperature=0,
                                 max_tokens=500):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,
        max_tokens=max_tokens,
    )
    return response.choices[0].message["content"]

In [ ]:
delimiter = "####"
system_message_1 = f"""
consider yourself as Human working on positive , negative or neural text classifier having great
deal of english language. Now given a text delimited with {delimiter} characters.
Highlight the parts of text which make it positive or negative or neutral.
Output should be in python dictionary format with keys being sentiment, part_of_text
and the values for sentiment and part_of_text should be single output string and should not have list of outputs for better understanding see examples bellow
where part_of_text contains only that words that have high similarity with the sentiments giving out different parts
of text which make it profane, vulgar, or offensive
Note that when sentiment is neutral, part_of_text simply contains user message and part_of_text contains only one index size

So, I need to train on these below examples
there will be 3 columns input_text,text_extracted,labels these columns will have values separated by comma
and their descriptions are as follows
input_text = actual text data
text_extracted = text which actually provides emotions to the original text
labels = labels/sentiment assigned to extracted text

1. you are stupid , stupid ,negative
2. Sooo SAD I will miss you here in San Diego!!!, Sooo SAD, negative
3. my boss is bullying me..., bullying me, negative
4. what interview! leave me alone, leave me alone, negative
5. Sons of bitch why couldn`t they put them on the releases we already bought, Sons of bitch, negative
6. y do i even bother getting a new fone..i just **** break them anyway fones dont float especially when thrown in the pool, **** break th, negative
7. The sun is attempting to increase her efforts. Alas the same cannot be said for me. I see a bath, a couch & a book in my immediate future, Alas,negative
8. Omg guys my internet has been down the whole day  It`s still not working  I`ll try fix 2m, Omg guys my internet has been down the whole day,negative
9. So sorry to hear your terrible news   Thinking of you all x, So sorry to hear your terrible news,negative
10. Shopped til i dropped....come bac sunshine i miss u, Shopped til i dropped....come bac sunshine i miss u,negative
11. there is absolutely nothing to eat in my house... epic fail, epic fail,negative
12. mom just woke me u[p and i am so mad i was dreaming about  shoes  she whants me to go to the river it is so stupid, o stupid,negative
13. So cold in here!  getting the chill , So cold in here!  getting the chill,negative
14. addin to that last comment, was spose to be going to see the jonas brothers 3D movie, but guess what i couldnt go aswell cos im sick, im sick,negative
15. I`m SO jealous- it`s pouring right now, jealous-,negative

16. A little happy for the wine jeje ok it`sm my free time so who cares jaja i love this day, A little happy fo, positive
17. lucky kid...i so wanna see loserville  pity im in oz...., lucky, positive
18. Happy Mothers day to all you Mums out there, Happy Mothers day to all you Mums out there, positive
19. Thank you, Afrin Nasal Spray! Also, I got a giant teacup tonight!, Thank you,, positive
20. Packing up for the trip to Ascari track, thank you Bacardi, thank you Bacardi, positive
21. Aha both same length, touche ,touche ,positive
22. I`m looking forward to going home tomorrow, but I really wish it was for a different reason., looking forward to,positive
23. _LOCA thank you babe  What are you doin this evening?, thank you ba,positive
24. Going home  but when i get home , BLACK BERRY ! WOO HOO  . . . and get to see presten ! <3, WOO HOO,positive
25. Working, Listening to music,Testing out a new singer/guitarist tonight, let`s hope for the best, hope,positive
26. its my birthday too  but its the 10th of may right now for me.. and its probably the 9th for you.. happy birthday anyway, . happy,positive
27. I am twittering, LIKE A BOSS. Thanks Savvv, Thanks Savvv,positive
28. Birthday Girl ?  blessed to live another year & celebrate with loved ones, blessed,positive
29. @__Jasper_Hale__ *Laughs* that`s because I like winning, like,positive
30. I love this!!  our body should tell us how much sleep it needs, and its always good to be mindful at mealtimes, s always good,positive

31. yeah, it`s a bit 80s silly at points but i love it, yeah, it`s a bit 80s silly at points but i love it, neutral
32. It's gonna be a super long day, It`s gonna be a super long day, neutral
33. I'm soooooooo hungry right now. DH is still not home, I`m soooooooo hungry right now. DH is still not home., neutral
34. Where`d the songs go on the site I want 'Do You' on this computer too, Where`d the songs go on the site I want 'Do You' on this computer too, neutral
35. fighting again, fighting again, neutral
36. omg i forgot about that! you lucky thing! hope you have fun!!  x, omg i forgot about that! you lucky thing! hope you have fun!!  x,neutral
37. . Really I look like a boy!!! I need someone to do my hair!, . Really I look like a boy!!! I need someone to do my hair!,neutral
38. _Cullen8 i hate the rain *laughs*. thanks *smiles* i miss her loads i couldnt stop crying before, _Cullen8 i hate the rain *laughs*. thanks *smiles* i miss her loads i couldnt stop crying before,neutral
39. Going to go kennel the dogs and then I am going to watch Goblet of Fire, Going to go kennel the dogs and then I am going to watch Goblet of Fire,neutral
40. later I went up to monroe and worked out with friends - TRX, ropes and KBs!!   30sec on/30 off..rope swings & c&ps, windmills, wve aquats, later I went up to monroe and worked out with friends - TRX, ropes and KBs!!   30sec on/30 off..rope swings & c&ps, windmills, wve aquats,neutral
41. _007 so I tried to send you a direct message and I can`t because you aren`t following me, 007 so I tried to send you a direct message and I can`t because you aren`t following me,neutral
42. Just watched the 'Final Break' final prison break episode. It was great. Farewell Prison Break. You will be dearly missed., Just watched the 'Final Break' final prison break episode. It was great. Farewell Prison Break. You will be dearly missed.,neutral
43. Im still in my my pjs  but I haveto get dressed cuz Im going to see Dancing on Ice live, Im still in my my pjs  but I haveto get dressed cuz Im going to see Dancing on Ice live,neutral
44. goodnight twitterville! really enjoyed pearl harbor. now off to sleep and cuddle with my hot man!  i will have good dreams tonight! XOXO, goodnight twitterville! really enjoyed pearl harbor. now off to sleep and cuddle with my hot man!  i will have good dreams tonight! XOXO,neutral
45. UGH... I am soooo drag assing today  but on a brighter note I got my G1 update & it is the biznesssss!, UGH... I am soooo drag assing today  but on a brighter note I got my G1 update & it is the biznesssss!,neutral
"""

In [4]:
from time import sleep
for j,i in enumerate(test.index):
    if i%3==0:
        sleep(60)
    user_message = good_test.iloc[i]['text']
    print(f'--->{user_message}\n')
    messages =  [
    {'role':'system',
     'content': system_message_1},
    {'role':'user',
     'content': f"{delimiter}{user_message}{delimiter}"},
    ]
    response = get_completion_from_messages(messages)
    try:
        response = json.loads(response.replace("'", '"'))
        print(j,response)
        print('\n')
    except json.JSONDecodeError as e:
        print("Error decoding JSON:", j,e)
        continue
    selected_text = response.get('part_of_text')
    selected_sentiment = response.get('sentiment')
#     print(selected_text)
    good_test['predicted_text'][j] = selected_text
    good_test['predicted_sentiment'][j] = selected_sentiment
#     break

In [ ]:
good_test_1 = good_test[0:219]
good_test_1 = good_test_1.dropna()
good_test_1

,textID,text,selected_text,sentiment,predicted_text,predicted_sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive,excited to see Samantha&Denise again,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative,doesn`t work on my Jaunty,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive,one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral,bored,neutral
...,...,...,...,...,...,...
214,8b3a2058c5,my cousin is in jail for shoplifting and drug...,im upset,negative,"cousin is in jail for shoplifting and drugs, u...",negative
215,d240f7c6d8,Good Morning Hows u ?,Good Morning,positive,Good Morning Hows u ?,neutral
216,aea00d01f6,i`m not being mean,i`m not being mean,positive,i`m not being mean,neutral
217,d5c1749c3a,my teeth and head hurts,my teeth and head hurts,negative,my teeth and head hurts,negative


In [ ]:
last_test = good_test_1.reset_index(drop=True)

In [ ]:
last_test

,textID,text,selected_text,sentiment,predicted_text,predicted_sentiment
0,33537ffd4d,is excited to see Samantha&Denise again http:...,excited,positive,excited to see Samantha&Denise again,positive
1,609bf831e2,apparently even Novell Moonlight doesn`t work ...,doesn`t work,negative,doesn`t work on my Jaunty,negative
2,4db58698d0,__ Aww bless her. She`s one of my faves,bless her. She`s one of my faves,positive,one of my faves,positive
3,fb45b19ca3,hahah I`ll try,hahah I`ll try,neutral,hahah I`ll try,neutral
4,630285dbc7,bored,bored,neutral,bored,neutral
...,...,...,...,...,...,...
207,8b3a2058c5,my cousin is in jail for shoplifting and drug...,im upset,negative,"cousin is in jail for shoplifting and drugs, u...",negative
208,d240f7c6d8,Good Morning Hows u ?,Good Morning,positive,Good Morning Hows u ?,neutral
209,aea00d01f6,i`m not being mean,i`m not being mean,positive,i`m not being mean,neutral
210,d5c1749c3a,my teeth and head hurts,my teeth and head hurts,negative,my teeth and head hurts,negative


In [ ]:
per_list = []
for j,i in enumerate(good_test.index):
    sen1 = last_test['selected_text'][i]
    sen2 = last_test['predicted_text'][i]

    percentage = Jaccard_Similarity(sen1, sen2)
    print(percentage)
    per_list.append(percentage)

0.2
0.4
0.5714285714285714
1.0
1.0
0.9523809523809523
0.2
0.2
0.2727272727272727
0.0
0.5
1.0
1.0
1.0
1.0
1.0
1.0
0.3333333333333333
0.38461538461538464
0.8
0.3333333333333333
0.3333333333333333
0.0
1.0
1.0
1.0
1.0
0.17647058823529413
1.0
0.1111111111111111
0.3181818181818182
1.0
1.0
0.07692307692307693
0.25
1.0
0.3157894736842105
1.0
0.34615384615384615
0.875
0.0
1.0
0.875
0.42857142857142855
0.1
1.0
0.6
0.125
0.2857142857142857
0.25
0.9333333333333333
0.18181818181818182
0.9166666666666666
0.25
1.0
0.125
1.0
0.0
0.6
1.0
0.0
0.6666666666666666
0.8333333333333334
0.75
1.0
1.0
0.08333333333333333
1.0
0.3333333333333333
1.0
0.5
0.8333333333333334
0.2
0.5833333333333334
0.2857142857142857
0.3333333333333333
0.2
0.25
1.0
1.0
0.42857142857142855
0.7142857142857143
0.4117647058823529
0.3076923076923077
0.9444444444444444
0.3333333333333333
1.0
1.0
0.1111111111111111
1.0
0.0
0.21428571428571427
0.6
1.0
0.391304347826087
0.35714285714285715
0.16
0.6
0.3157894736842105
1.0
0.32
0.125
0.5
0.04166

KeyError: 212

### Accuracy (when we use 45 statements in prompt)

In [ ]:
average = sum(per_list) / len(per_list)
print(average)

0.5602548798099986


### Save File

In [ ]:
# last_test.to_csv('/home/user/Desktop/Extra/56_score_data.csv', index=False)